In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# ENABLE IF USING COLAB
USE_COLAB = False

if USE_COLAB:
    import os
    import shutil
    if os.path.exists('/content/spectroconv/'):
        shutil.rmtree('/content/spectroconv/') #deletes a directory and all its contents.
    !git clone https://github.com/jakeval/spectroconv.git
    %cd /content/spectroconv
    !git checkout cnn-debug
    %cd ..
    !pip install hub
    !pip install hub[audio]
    !pip install wandb
    import sys
    sys.path.insert(0,'/content/spectroconv')

In [3]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: nvidia-smi: command not found


In [16]:
from models import wb_experiment

In [7]:
n_mels = 128
wb_config = {
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'group': 'cnn' # use this only when training/evaluating/etc a CNN
}

config = {
    'data': {
        'train': {
            'name': 'nsynth-tiny-shift-train'
        },
        'val': {
            'name': 'nsynth-tiny-shift-val'
        }
    },
    'model': {
        'id': wb_experiment.ModelType.CNN.value,
        'name': 'CNN-tiny'
    },
    'logging': {
        'batch_log_interval': 50,
        'eval_every_epoch': True,
    },
    'parameters': {
        'epochs': 2,
        'batch_size': 64,
        'optimizer': 'adam',
        'learning_rate': 0.0005,
        'dropout_fc': 0.5, # probability of an element to be zeroed (0 = no dropout)
        'dropout_conv': 0.1,
        'dropout_input': 0.2,
        'l2_reg': 0,
        'kernel_size': 3,
        'num_channels': [8, 16],
        'linear_layer_size': 32,
        'max_norm': 0.75,
    },
}

In [9]:
trainer = wb_experiment.TrainExperiment(wb_config=wb_config, login_key=False)
model = trainer.run_train(config, save_model=True)

hub://jakeval/nsynth-tiny-shifttrain-metadata loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-shifttrain-metadata.
hub://jakeval/nsynth-tiny-shifttrain loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-shifttrain.
hub://jakeval/nsynth-tiny-shiftval-metadata loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-shiftval-metadata.
hub://jakeval/nsynth-tiny-shiftval loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-tiny-shiftval.


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

Batch 0: Loss: 0.6998, Accuracy: 0.3125


  0%|          | 0/4 [00:00<?, ?it/s]

Batch 0: Loss: 0.0553, Accuracy: 0.9909



train/accuracy,▁█
train/f1,▁█
train/loss,█▁
val/accuracy,▁▁
val/f1,▁▁
val/loss,▁█
train/accuracy,0.99091
train/f1,0.9909
train/loss,0.05527
val/accuracy,0.53947
val/f1,0.35033


In [19]:
# untested

config = {
    'data': {
        'val': {
            'name': 'nsynth-full-128-val'
        },
        'train': {
            'name': 'nsynth-full-128-train'
        }
    },
    'model_name': 'CNN-full',
    'eval': {
        'examples': {
            'k': 8
        }
    }
}


In [ ]:
trainer = wb_experiment.EvalExperiment(wb_config)
metrics, examples_df, model_stats = trainer.run_evaluate(config)

hub://jakeval/nsynth-full-128-val-metadata loaded successfully.


/home/jakeval/umass/682/project/spectroconv/.env/lib/python3.8/site-packages/hub/core/fast_forwarding.py:43: UserWarning: Loading a dataset that was created or updated with a newer version of hub. This could lead to corruption or unexpected errors! Dataset version: 2.4.0, current hub version: 2.3.2. It's recommended that you update to a version of hub >= 2.4.0.
  warnings.warn(


This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-full-128-val-metadata.
hub://jakeval/nsynth-full-128-val loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-full-128-val.
hub://jakeval/nsynth-full-128-train-metadata loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-full-128-train-metadata.
hub://jakeval/nsynth-full-128-train loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/jakeval/nsynth-full-128-train.
Loading 8 spectrograms...
Loading 8 audio clips...
Opening dataset in read-only mode as you don't have write permissions.
hub://activeloop/nsynth-val loaded successfully.
This dataset can be visualized at https://app.activeloop.ai/activeloop/nsynth-val.
Linished loading
Loading 8 spectrograms...
Loading 8 audio clips...
Opening dataset in read-only mode as you don't have write permissions.
hub://activeloop/nsynth-val loaded successfully.
Th

## Train on the full dataset

In [ ]:
# Wandb should take care of plotting for us?

'''
from matplotlib import pyplot as plt
fig, ax = plt.subplots()
ax.plot(losses)
fig, ax = plt.subplots()
ax.plot(train_accuracies, label='train acc')
ax.plot(val_accuracies, label='val acc')
ax.plot(val_f1s, label='val f1')
ax.legend()
plt.savefig("learning_metrics.png")
plt.show()
'''

In [ ]:
#print(train_model.stream_accuracy(clf, dl_val, device))

0.7415788789179347


In [ ]:
#print(train_model.stream_f1(clf, dl_val, device))

0.7109029348873568
